In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
import transfer_func_util as tf
import configuration as config

### Parameters

In [7]:
## some are coming from the notebook: Old trap - 20211001 - All calibration with comments
bandwidth = 2
decimate = 10
drive_freq = 139
fsamp = 5000

scaleZ = 128.04188971089127

voltage_calib = 24.0265126779

E1, E2 = (131, 196)
E_tot = E1+E2
factor = 0.6188

In [8]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_200 = np.std(xx2)*np.sqrt(2)*100
   


In [9]:
etha = 1/scaleZ*E_tot*voltage_calib*1.6e-19 ## convert bits to force
to_epsilon = 1/(2*131*1.6e-19*voltage_200) ## this is 2E1 - the convertion factor from A to epsilon

### Look at neutrality data

In [10]:
def get_amp_wraperZ(i, fname):
    try:
        return get_ampZ(i, fname)
    except:
        return 0,0,0

In [11]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_ampZ(i, foldername):
    
    fname = foldername+'Z_200Vpp_'+str(i)+'.h5'
    bb.load(fname);
    bb.load_other_data()
    
    ## extract the drive phase - depends on which electrode is driving
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    ## bandpass and fit
    phase_diff = 4.8        
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma': [100,10000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    ## bandpass and fit sideband
    phase_diff = 4.8        
    fit_kwargs = {'A': 0, 'f': drive_freq+1, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma': [100,10000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}
    b, a = signal.butter(3, [2.*(drive_freq+1-bandwidth/2.)/fsamp, 2.*(drive_freq+1+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m1_tmp.values[3], m2_tmp.values[0]

In [12]:
base_folder = r'/data/old_trap/20211001/bead1/'
fname = base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/'
get_ampZ(0, fname)

(-9.674676215377985, 1472.9405515946455, 0.008885914226993918)

### Loading all datasets taken with ~100kHz spinning field 

In [13]:
## the prefix of the some of the folders is Z_150Vpp, but that is raelly 200V data
## I messed up the the folder names, it is spinning at 175Vpp with 200Vpp oscillating field 
## (it has been validated using the digitized electrode data and also by the filenames which are 200Vpp)
## spinning is either 93kHz or 103kHz
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/',
base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb/',
base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo/',
# base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo2/', ## rga incident
base_folder+r'neutrality6/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb_redo3/',
base_folder+r'neutrality6/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb_redo4/']

files = [2000, 2000, 2000, 1000, 1000]

In [14]:
ams_neutrality6 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality6 += list(np.array(amps_tmp)*etha)
ams_neutrality6 = np.array(ams_neutrality6)

  4%|▍         | 80/2000 [00:00<00:03, 595.77it/s]/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 1000/1000 [00:16<00:00, 62.07it/s]


In [15]:
## getting rid of one corrupted file - have to extract two files in order to keep the odd always plus electrode
ams_neutrality6_2 = list(ams_neutrality6)
ams_neutrality6_2.pop(692)
ams_neutrality6_2.pop(692)
ams_neutrality6 = np.array(ams_neutrality6_2)

In [16]:
## higher spinning - 117kHz
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_5sec_with_height_fb/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo2/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo3/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo4/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo5/']

files = [2000, 2000, 2000, 2000, 438]

In [17]:
ams_neutrality8 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality8 += list(np.array(amps_tmp)*etha)
ams_neutrality8 = np.array(ams_neutrality8)

100%|██████████| 438/438 [00:05<00:00, 80.94it/s] 


In [18]:
## higher spinning - 117kHz
## after improving the switching on/off of the electrodes to allow more off time per configuration
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo2/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo3/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_6sec_with_height_fb_redo4/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_6sec_with_height_fb_redo6/']

files = [80, 2000, 2000, 2000, 2000, 4000, 2200]

In [19]:
ams_neutrality9 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality9 += list(np.array(amps_tmp)*etha)
ams_neutrality9 = np.array(ams_neutrality9)

100%|██████████| 2200/2200 [00:35<00:00, 55.72it/s]


In [20]:
# different spin voltage -- skipped that one accidently before it adds a bit of sensitity
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality11/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_200Vpp_delay_6sec_with_height_fb/']

files = [2500]

In [21]:
ams_neutrality11 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality11 += list(np.array(amps_tmp)*etha)
ams_neutrality11 = np.array(ams_neutrality11)

100%|██████████| 2500/2500 [00:47<00:00, 52.63it/s]


In [22]:
amps_all = np.array(list(ams_neutrality6)+list(ams_neutrality8)+list(ams_neutrality9)+list(ams_neutrality11))

### Data Analysis

In [23]:
data_plus = amps_all[::2,0]
data_minus = amps_all[1::2,0]
data_A = data_plus-factor*data_minus

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data_plus))*2*10/3600, data_plus, label='200Vpp-z-plus')
ax.scatter(np.arange(len(data_minus))*2*10/3600, data_minus, label='200Vpp-z-minus', alpha=0.6)
ax.plot(np.arange(len(data_A))*2*10/3600, np.arange(len(data_A))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, electrode 1: mean, std: ', np.mean(data_plus), np.std(data_plus)/np.sqrt(len(data_plus)))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data_minus), np.std(data_minus)/np.sqrt(len(data_minus)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data_A), np.std(data_A)/np.sqrt(len(data_A)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -2.31010057811e-17 6.33739840828e-19
1st harmonic, electrode 2: mean, std:  -3.76532518346e-17 8.23607072671e-19
1st harmonic, A parameter: mean, std:  1.98826454112e-19 4.0224333327e-19


In [24]:
len(data_plus)*2*10/3600

92.26666666666667

In [25]:
N = len(data_plus)
data_shifted = np.array([(data_plus[i]+data_plus[i-1])/2 for i in range(1,N)])
data_A_shifted = data_shifted-factor*data_minus[:-1]

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data_A))*2*10/3600, data_A, label='200Vpp-z-plus, A parameter')
ax.scatter(np.arange(len(data_A_shifted))*2*10/3600, data_A_shifted, label='200Vpp-z, corrected A parameter', alpha=0.6)

ax.plot(np.arange(len(data_A))*2*10/3600, np.arange(len(data_A))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, A parameter: mean, std: ', np.mean(data_A), np.std(data_A)/np.sqrt(len(data_A)))
print('1st harmonic, corrected A: mean, std: ', np.mean(data_A_shifted), np.std(data_A_shifted)/np.sqrt(len(data_A_shifted)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data_A_shifted)*to_epsilon, to_epsilon*np.std(data_A_shifted)/np.sqrt(len(data_A_shifted)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, A parameter: mean, std:  1.98826454112e-19 4.0224333327e-19
1st harmonic, corrected A: mean, std:  2.00454777895e-19 3.2205906766e-19
1st harmonic, A parameter[epsilon]: mean, std:  2.50526043314e-05 4.02505666273e-05


In [26]:
## sideband
data_plus2 = amps_all[::2,2]
data_minus2 = amps_all[1::2,2]
data_A2 = data_plus2-factor*data_minus2
N = len(data_plus2)
data_shifted2 = np.array([(data_plus2[i]+data_plus2[i-1])/2 for i in range(1,N)])
data_A_shifted_noise = data_shifted2-factor*data_minus2[:-1]

In [31]:
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data_A_shifted)*to_epsilon, to_epsilon*np.std(data_A_shifted)/np.sqrt(len(data_A_shifted)))
print('sideband, A parameter[epsilon]: mean, std: ', np.mean(data_A_shifted_noise)*to_epsilon, to_epsilon*np.std(data_A_shifted_noise)/np.sqrt(len(data_A_shifted_noise)))

1st harmonic, A parameter[epsilon]: mean, std:  2.50526043314e-05 4.02505666273e-05
sideband, A parameter[epsilon]: mean, std:  -9.97051721328e-05 3.83254646891e-05


In [32]:
# epsilon_array = np.array([data_A_shifted, data_A_shifted_noise])*to_epsilon
# np.save("epslion_old_trap_20210812", epsilon_array)

### Fitting

In [35]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 29
range_pram = 0.02
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_A_shifted*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), label='200Vpp, all frequencies')
ax.set(xlabel=r'Force [N]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Force [N]')]

In [36]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.03       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], " ,std (on the mean): ", popt[2]/np.sqrt(len(data_A_shifted)), " ,sigma: ", popt[2])
    print("std from the covariance matrix: ", np.sqrt(pcov[1,1]))
    
    nucleons = 420e-12*6e23/2
    print('\nNeutrality: ', 1/nucleons*popt[2]/np.sqrt(len(data_A_shifted)))
    print('Factor needed for neutrality best limit: ', 1/nucleons*popt[2]/np.sqrt(len(data_A_shifted))/1e-21)

mean:  2.20881613977e-05  ,std (on the mean):  -3.98018381151e-05  ,sigma:  -0.00512918923145
std from the covariance matrix:  3.92099676968e-05

Neutrality:  -3.15887604088e-19
Factor needed for neutrality best limit:  -315.887604088
